# Специальные методы классов

In [18]:
import random

class Vector:
    def __init__(self, x=0, y=0, color=None):
        print("initializing a vector")
        if type(x) != int or type(y) != int:
            raise AttributeError('x and y should be int')
        
        self._x = x
        self._y = y
        self._color = color
    
    def get_x(self):
        return self._x
    
    def get_y(self):
        return self._y

Методы с двойным подчеркиванием в начале и конце имени имеют особое значение. 

Мы уже знакомы с `__next__` и `__iter__`, пора узнать и об остальных

In [19]:
vector = Vector(1, 2, 'red')
print(vector)
str(vector) # --> vector.__str__()

initializing a vector


'<__main__.Vector object at 0x7f653c5e2d68>'

In [20]:
vector.__str__()

'<__main__.Vector object at 0x7f653c5e2d68>'

In [21]:
class VectorWithStr(Vector):
    def __str__(self):
        return 'vector ({}, {}) of color {}'.format(self._x, self._y, self._color)

In [22]:
vector = VectorWithStr(1, 2, 'red')
str(vector)

initializing a vector


'vector (1, 2) of color red'

__Q:__ Преобразование в строку? Это всё?

__A:__ Конечно, нет. Неявные преобразования иногда происходят там, где мы их не ожидаем

In [23]:
print(vector)

vector (1, 2) of color red


In [24]:
print("OBJECT: {}".format(vector))

OBJECT: vector (1, 2) of color red


In [29]:
mydict = {}
mydict[vector] = 2
mydict

{<__main__.VectorWithStr at 0x7f653c5e2e10>: 2}

In [30]:
mylist = [vector]
print(mylist)

__Q:__ А откуда опять "некрасивые" строки?!

__A:__ В питоне используется два способа приведения к строке. Это функции `str` и `repr`, которые отличаются своим назначением. 

`str` используется там, где нужна человекочитаемость, а `repr` реализуется так, чтобы можно было однозначно определить, о каком объекте идет речь. Если `repr` не реализован, используется стандартный вариант, а если не реализован `str`, то вместо него используется `repr`. 

Попробуем?

In [31]:
class VectorWithRepr(Vector):
    def __repr__(self):
        return 'vector representation (x: {}, y: {}, color: {})'.format(self._x, self._y, self._color)

In [34]:
vector = VectorWithRepr(1, 2, 'red')

print(vector)
mylist = [vector]
print(mylist)
mydict = {}
mydict[vector] = 2
print(mydict)

initializing a vector
vector representation (x: 1, y: 2, color: red)
[vector representation (x: 1, y: 2, color: red)]
{vector representation (x: 1, y: 2, color: red): 2}


In [35]:
class VectorWithBothReprAndStr(VectorWithRepr, VectorWithStr):
    pass

In [37]:
vector = VectorWithBothReprAndStr(1, 2, 'red')
# вот здесь должны получиться разные значения
print(vector)
print([vector])

initializing a vector
vector (1, 2) of color red
[vector representation (x: 1, y: 2, color: red)]


## Арифметика

In [43]:
import math
import random

class VectorWithMath(VectorWithBothReprAndStr):    
    def __abs__(self):
        return math.hypot(self._x, self._y)
    
    def __add__(self, other): # vector1 + vector2
        print('self={}, other={}'.format(id(self), id(other)))
        return VectorWithMath(self.get_x() + other.get_x(),
                     self.get_y() + other.get_y(),
                     random.choice((str(self._color), str(other._color))))
    
    def __sub__(self, other): # vector1 - vector2
        return VectorWithMath(self.get_x() - other.get_x(),
                     self.get_y() - other.get_y(),
                     random.choice((str(self._color), str(other._color))))
    
    # ещё есть div, mul и многое другое
    # vector += vector2 - __iadd__ etc
    # __add__   a + b -->  a.__add__(b)
    # __radd__   a + b --> b.__add__(a)

In [44]:
vector1 = VectorWithMath(3, 4, 'blue')
vector2 = VectorWithMath(1, 2, 'red')

initializing a vector
initializing a vector


In [45]:
print(abs(vector1))
print(vector1 + vector2)
print(vector2 + vector1)

5.0
self=140072780235272, other=140072780235160
initializing a vector
vector (4, 6) of color red
self=140072780235160, other=140072780235272
initializing a vector
vector (4, 6) of color blue


## Приведение типов

In [46]:
import math

class VectorWithTypes(VectorWithMath):
    def __bool__(self):
        print('__bool__')
        return bool(self._x) or bool(self._y)
    
    def __int__(self):
        print('__int__')
        return int(float(self))
    
    def __float__(self):
        print('__float__')
        return abs(self)

In [48]:
def returns_int() -> int:
    return 0

In [49]:
help(returns_int)

Help on function returns_int in module __main__:

returns_int() -> int



In [47]:
vector = VectorWithTypes(3, 4, 'blue')
print(vector)
print(int(vector))
print(float(vector))
if vector:
    print("vector ~ True")

initializing a vector
vector (3, 4) of color blue
__int__
__float__
5
__float__
5.0
__bool__
vector ~ True


In [50]:
vector = VectorWithTypes()
print(vector)
if not vector:
    print("vector ~ False")

initializing a vector
vector (0, 0) of color None
__bool__
vector ~ False


## Итерирование

Один способ сделать объект "итерабельным" нам уже известен, это метод `__next__`. Но он не единственный

In [52]:
class VectorIterable(VectorWithTypes):
    def __getitem__(self, position):
        return (self._x, self._y)[position]
    
    def __len__(self):
        return 2
    
    def __reversed__(self):
        return (self._x, self._y)[::-1]

In [54]:
vector = VectorIterable(100, 500)
print(vector[0])
vector[3]

initializing a vector
100


IndexError: tuple index out of range

In [55]:
for coordinate in vector:
    print(coordinate)

100
500


In [56]:
for coordinate in reversed(vector):
    print(coordinate)

500
100


## Динамическая работа с атрибутами

Казалось бы, в питоне нет никакой защиты от "взлома". Но нельзя ли сделать её самостоятельно?

In [60]:
class VectorWithAllAttributes(VectorIterable):
    def __getattr__(self, attr_name):
        print('__getattr__')
        return "value of {}".format(attr_name)
    
    def __getattribute__(self, attr_name):
        print('__getattribute__')
        return super().__getattribute__(attr_name)
    
    def __setattr__(self, attr_name, attr_value):
        if attr_name not in ('_x', '_y', '_color'):
            raise Exception('you shall not add new attributes here, young padawan!')
        else:
            super().__setattr__(attr_name, attr_value)
            
    def __delattr__(self, attr_name):
        print('Heh, you can delete nothing')

In [63]:
vector = VectorWithAllAttributes(1, 2, 'violet')
print(dir(vector))

initializing a vector
__getattribute__
__getattribute__
['__abs__', '__add__', '__bool__', '__class__', '__delattr__', '__dict__', '__dir__', '__doc__', '__eq__', '__float__', '__format__', '__ge__', '__getattr__', '__getattribute__', '__getitem__', '__gt__', '__hash__', '__init__', '__int__', '__le__', '__len__', '__lt__', '__module__', '__ne__', '__new__', '__reduce__', '__reduce_ex__', '__repr__', '__reversed__', '__setattr__', '__sizeof__', '__str__', '__sub__', '__subclasshook__', '__weakref__', '_color', '_x', '_y', 'get_x', 'get_y']


In [70]:
vector.__dict__, VectorWithAllAttributes.__dict__

__getattribute__


({'_color': 'violet', '_x': 1, '_y': 2},
 mappingproxy({'__delattr__': <function __main__.VectorWithAllAttributes.__delattr__(self, attr_name)>,
               '__doc__': None,
               '__getattr__': <function __main__.VectorWithAllAttributes.__getattr__(self, attr_name)>,
               '__getattribute__': <function __main__.VectorWithAllAttributes.__getattribute__(self, attr_name)>,
               '__module__': '__main__',
               '__setattr__': <function __main__.VectorWithAllAttributes.__setattr__(self, attr_name, attr_value)>}))

In [65]:
print(vector.some_attribute)
print(vector._color)
# print(vector.get_x())

__getattribute__
__getattr__
value of some_attribute
__getattribute__
violet


In [66]:
vector.new_attribute = "value"

Exception: you shall not add new attributes here, young padawan!

In [71]:
delattr(vector, '_color')
print(vector._color)

Heh, you can delete nothing
__getattribute__
violet


## Контексты

In [73]:
file = open('file.txt', 'w')
file.write('Hello, world')
file.close()

In [80]:
file = open('file.txt', 'r')
file.read()
print(file.closed)
file.close()
print(file.closed)

False
True


In [81]:
with open('file.txt', 'r') as f:
    f.read()
f.closed

True

In [82]:
class VectorWithContextManager(VectorWithAllAttributes):
    def __enter__(self):
        print('entering context')
    def __exit__(self, *args):
        print(args)
        print('leaving context')

In [83]:
try:
    with VectorWithContextManager() as vec:
        for i in range(3):
            print(i)
        raise Exception('something happened inside!')
except Exception:
    print('an exception was raised...')
    pass
print('we are out if the context')

initializing a vector
entering context
0
1
2
(<class 'Exception'>, Exception('something happened inside!',), <traceback object at 0x7f653c4f8c48>)
leaving context
an exception was raised...
we are out if the context


Но можно и проще!

In [85]:
from contextlib import contextmanager

@contextmanager
def vector_mgr():
    print('handling entering the context')
    yield Vector()
    print('handling leaving the context')
          
print('statement before context')
with vector_mgr() as vector:
    print(vector)
print('statement after context')

statement before context
handling entering the context
initializing a vector
handling leaving the context
statement after context


In [92]:
vector_mgr().func()

<generator object vector_mgr at 0x7f653c4fad58>

In [88]:
dir(vector_mgr())

['__call__',
 '__class__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__enter__',
 '__eq__',
 '__exit__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__gt__',
 '__hash__',
 '__init__',
 '__le__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 '_recreate_cm',
 'args',
 'func',
 'gen',
 'kwds']

## Создание и удаление объектов

In [93]:
class VectorInitialized(VectorWithContextManager):
    def __new__(cls, *args, **kwargs):
        print('invoking __new__ method')
        print(cls, args, kwargs)
        return object.__new__(cls)
    
    def __del__(self):
        print('deleting an object')
        raise Exception("exception while destructing")

In [94]:
vector = VectorInitialized(1, 2, color='navy blue')

invoking __new__ method
<class '__main__.VectorInitialized'> (1, 2) {'color': 'navy blue'}
initializing a vector


In [95]:
del vector

deleting an object
__getattribute__
__getattribute__
__getattribute__


Exception ignored in: <bound method VectorInitialized.__del__ of vector representation (x: 1, y: 2, color: navy blue)>
Traceback (most recent call last):
  File "<ipython-input-93-d136fc659a36>", line 9, in __del__
Exception: exception while destructing


In [96]:
vector

NameError: name 'vector' is not defined

### Упражнение! 

Как с помощью метода `__new__` сделать класс "синглтоном" -- объектом, который создается один раз, а при попытке повторного создания возвращается уже готовый объект?

In [110]:
class SingletonClassBase:
    def __new__(cls, *args, **kwargs):
        print(cls)
        return super().__new__(cls, *args, **kwargs)

class SingletonClass(SingletonClassBase):
#     ...your code here...
    obj = None
    def __new__(cls, *args, **kwargs):
#         ...your code here...
        print(cls)
        if cls.obj is None:
            cls.obj = super().__new__(super(), *args, **kwargs)
        return cls.obj

obj1 = SingletonClass()
obj2 = SingletonClass()
assert id(obj1) == id(obj2)

<class '__main__.SingletonClass'>
<super: <class 'SingletonClass'>, <SingletonClass object>>


TypeError: super(type, obj): obj must be an instance or subtype of type

In [108]:
id(obj1), id(None)

(140072780353832, 10748000)